In [25]:
import pandas as pd
import cdsapi
import zipfile
from os.path import exists

file = pd.read_csv("CMIP6-table-for-Amine.csv", sep=";")
file.head()

,Time resolution,Experience,Variable,Model,Availability
0,monthly,historical,surface_downwelling_shortwave_radiation,cesm2,Yes
1,monthly,historical,surface_downwelling_shortwave_radiation,cesm2_waccm,Yes
2,monthly,historical,surface_downwelling_shortwave_radiation,cnrm_cm6_1_hr,Yes
3,monthly,historical,surface_downwelling_shortwave_radiation,mri_esm2_0,Yes
4,monthly,historical,surface_downwelling_shortwave_radiation,ukesm1_0_ll,Yes


In [30]:
def extract_criteria():
    file = pd.read_csv("CMIP6-table-for-Amine.csv", sep=";")
    c = cdsapi.Client()
    for tres in file["Time resolution"]:
        print(tres)
        for exp in file["Experience"]:
            for var in file["Variable"]:
                for mod in file["Model"]:
                    newFile = tres+exp+var+mod+".zip"
                    if not exists(newFile):
                        # the year range differ for historical;
                        if exp == "historical":
                            try:
                                c.retrieve("projections-cmip6", 
                                   {"temporal_resolution": tres,
                                   "experiment": exp,
                                   "variable": var,
                                   "model": mod,
                                    "year": [str(year) for year in range(2000, 2014)],
                                    "month": [str(month).zfill(2) for month in range(1, 13)],
                                    "day": [str(day).zfill(2) for day in range(1, 32)],
                                    # CHECK THE COORDINATES;
                                    "area": [-30, -59, -35, -53],
                                    "format": "zip"
                                   }, newFile)
                                with zipfile.Zipfile(newFile, "r") as zip_f:
                                    zip_f.extractall()
                            except:
                                print(newfile + " cannot be found.")
                        else:
                            c.retrieve("projections-cmip6", 
                               {"temporal_resolution": tres,
                               "experiment": exp,
                               "variable": var,
                               "model": mod,
                                "year": [str(year) for year in range(2015, 2051)],
                                "month": [str(month).zfill(2) for month in range(1, 13)],
                                "day": [str(day).zfill(2) for day in range(1, 32)],
                                # CHECK THE COORDINATES;
                                "area": [-30, -59, -35, -53],
                                "format": "zip"
                               }, newFile)
                            with zipfile.Zipfile(newFile, "r") as zip_f:
                                    zip_f.extractall()

In [36]:
c = cdsapi.Client()
newFile = "example.zip"
c.retrieve("projections-cmip6", 
    {"temporal_resolution": "monthly",
     "experiment": "historical",
     "variable": "surface_downwelling_shortwave_radiation",
     "model": "cesm2",
     "year": [str(year) for year in range(2000, 2015)],
     "month": [str(month).zfill(2) for month in range(1, 13)],
     "day": [str(day).zfill(2) for day in range(1, 32)],
     # CHECK THE COORDINATES;
     "area": [-30, -59, -35, -53],
     "format": "zip"
     }, newFile)
with zipfile.Zipfile(newFile, "r") as zip_f:
    zip_f.extractall()

2024-03-28 18:31:23,361 INFO Welcome to the CDS
2024-03-28 18:31:23,363 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-03-28 18:31:23,418 INFO Request is queued
2024-03-28 18:31:24,475 INFO Request is running
2024-03-28 18:31:36,792 INFO Request is completed
2024-03-28 18:31:36,793 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data9/adaptor.esgf_wps.retrieve-1711647091.2346177-9025-14-21e515f3-39ff-486d-ad7b-94ca517779f6.zip to example.zip (257.9K)
2024-03-28 18:31:37,045 INFO Download rate 1M/s                                 


AttributeError: module 'zipfile' has no attribute 'Zipfile'